---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [32]:
def date_sorter():
    
    # Your code here

    remaining_indexes = list(df.index[:])

    # mm/dd/yy | mm-dd-yy type
    type1 = df[remaining_indexes].str.extractall(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2})\b')
    type1['month'] = type1['month'].astype(int)
    type1['day'] = type1['day'].astype(int)
    type1['year'] = ('19' + type1['year']).astype(int)
    type1.reset_index(inplace=True)
    for x in list(type1['level_0']):
        remaining_indexes.remove(x)

    # mm/dd/yyyy | mm-dd-yyyy type
    type2 = df[remaining_indexes].str.extractall(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{4})\b')
    type2['month'] = type2['month'].astype(int)
    type2['day'] = type2['day'].astype(int)
    type2['year'] = type2['year'].astype(int)
    type2.reset_index(inplace=True)
    for x in list(type2['level_0']):
        remaining_indexes.remove(x)

    # mmm-dd-yyyy | mmm dd, yyyy | mmm. dd, yyyy | mmm dd yyyy type
    type3 = df[remaining_indexes].str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[-.]* (?P<day>\d{1,2})[- ,]* +(?P<year>\d{4})')
    type3['day'] = type3['day'].astype(int)
    type3['year'] = type3['year'].astype(int)
    type3.reset_index(inplace=True)
    for x in list(type3['level_0']):
        remaining_indexes.remove(x)

    # dd mmm yyyy | dd mmm. yyyy | dd mmm, yyyy type
    type4 = df[remaining_indexes].str.extractall(r'(?:(?P<day>\d{1,2}) )(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.]* (?P<year>\d{4})')
    type4['day'] = type4['day'].astype(int)
    type4['year'] = type4['year'].astype(int)
    type4.reset_index(inplace=True)
    for x in list(type4['level_0']):
        remaining_indexes.remove(x)

    # mmm ddxx, yyyy type
    type5 = df[remaining_indexes].str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)) (?P<day>\d{1,2})[a-z]{2}[,]{1} (?P<year>\d{4})')
    type5['day'] = type5['day'].astype(int)
    type5['year'] = type5['year'].astype(int)
    type5.reset_index(inplace=True)
    for x in list(type5['level_0']):
        remaining_indexes.remove(x)

    # mmm yyyy type
    type6 = df[remaining_indexes].str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[,.]*(?P<day>) (?P<year>\d{4})\b')
    type6['day'] = 1
    type6['year'] = type6['year'].astype(int)
    type6.reset_index(inplace=True)
    for x in list(type6['level_0']):
        remaining_indexes.remove(x)

    # mm/yyyy type
    type7 = df[remaining_indexes].str.extractall(r'(?P<month>\d{1,2})[/](?P<day>)(?P<year>\d{4})')
    type7['month'] = type7['month'].astype(int)
    type7['day'] = 1
    type7['year'] = type7['year'].astype(int)
    type7.reset_index(inplace=True)
    for x in list(type7['level_0']):
        remaining_indexes.remove(x)

    # yyyy type
    type8 = df[remaining_indexes].str.extractall(r'(?P<month>)(?P<day>)(?P<year>\d{4})\b')
    type8['month'] = 1
    type8['day'] = 1
    type8['year'] = type8['year'].astype(int)
    type8.reset_index(inplace=True)
    for x in list(type8['level_0']):
        remaining_indexes.remove(x)

    total = pd.DataFrame()
    total = total.append(type1)
    total = total.append(type2)
    total = total.append(type3)
    total = total.append(type4)
    total = total.append(type5)
    total = total.append(type6)
    total = total.append(type7)
    total = total.append(type8)

    #month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
    #                   'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
    #                   'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
    #                   'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})

    month_dict = dict({'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 
                        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12, 'January': 1, 'February': 2, 'March': 3, 
                        'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 
                        'October': 10, 'November': 11, 'December': 12, 'Janaury': 1, 'Decemeber': 12})

    total.replace({'month': month_dict}, inplace=True)
    #total.replace(to_replace=['Janaury'], value='January', inplace=True)
    #return total[450:]

    total['date'] = pd.to_datetime(total['month'].apply(str) + '/' + total['day'].apply(str) + '/' + total['year'].apply(str))
    #return 
    return total.sort_values('date')['level_0'].astype(int)

date_sorter()

9       9
67     84
2       2
44     53
24     28
     ... 
84    427
16    141
61    186
36    161
70    413
Name: level_0, Length: 500, dtype: int64